In [1]:
# Preparar Dataset, agrupando info por minuto
#  e importando desde Google Ethereum BigQuery

In [2]:
# Incluir definición de parámetros
%run -i '/content/drive/MyDrive/TFM/0.config.py'

Usando DS_FILE       : /content/drive/MyDrive/TFM/etherdata-MINUTE-2021-09-01-2021-11-30.csv
Usando DS_FILE_CLEAN : /content/drive/MyDrive/TFM/etherdata-MINUTE-2021-09-01-2021-11-30-CLEAN.csv


In [3]:
# Importaciones
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
# Autenticar en Google Cloud
# Debe existir un proyecto PROJECT con billing activado, tal como se describe en 
#   https://colab.research.google.com/notebooks/bigquery.ipynb
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [5]:
# Crear cliente biggquey
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT)

In [6]:
# Preparar query sobre Ethereum BigQuery
#   anidando busqueda en bloques y transacciones 
#   y agrupando por unidades de tiempo
query = """
SELECT 

  AVG(base_fee_per_gas/POWER(10,9)) AS avg_base_fee_per_gas,  
  AVG(difficulty)                   AS avg_difficulty,
  AVG(transaction_count)            AS avg_transaction_count,
  AVG(gas_limit)                    AS avg_gas_limit,
  AVG(gas_used)                     AS avg_gas_used,
  AVG(size/POWER(10,3))             AS avg_size,
  COUNT (*)                         AS num_blk,
  """+TIME_AGREGATION+"""           AS time,

  AVG(trns.avg_trn_gas_price)       AS avg_gas_price,      
  MIN(trns.min_trn_gas_price)       AS min_gas_price,      
  MAX(trns.max_trn_gas_price)       AS max_gas_price,      
  SUM(trns.sum_trn_tx_ether)        AS sum_tx_ether,                    
  SUM(trns.num_trn_blk)             AS num_trn,                    

FROM
  `bigquery-public-data.crypto_ethereum.blocks` AS blocks
   LEFT JOIN 
      (SELECT 
        block_number,
        AVG(gas_price/POWER(10,9))     AS avg_trn_gas_price,
        MIN(gas_price/POWER(10,9))     AS min_trn_gas_price,
        MAX(gas_price/POWER(10,9))     AS max_trn_gas_price,
        SUM(value/POWER(10,18))        AS sum_trn_tx_ether,
        COUNT (*)                      AS num_trn_blk,
      FROM 
        `bigquery-public-data.crypto_ethereum.transactions` 
      GROUP BY block_number) AS trns 
   ON blocks.number = trns.block_number
GROUP BY time
HAVING DATE(time) >= '"""+DATE1+"""' AND DATE(time) <= '"""+DATE2+"""'
ORDER BY time
"""

In [7]:
# Ejecutar query
query_job = client.query(query)
iterator = query_job.result(timeout=60)
rows = list(iterator)

# Transform the rows into a pandas dataframe
import pandas as pd
df = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))

# Covertir difficulty de  tipo object a float
cols = ['avg_difficulty']
df[cols] = df[cols].apply(pd.to_numeric, downcast='float', errors='coerce')

In [8]:
# Ordenar columnas
df = df[['time','avg_gas_price','min_gas_price','max_gas_price','avg_base_fee_per_gas','avg_difficulty','avg_gas_limit','avg_gas_used','avg_size','avg_transaction_count','sum_tx_ether','num_blk','num_trn']]

In [9]:
# Guardar Dataset en Google Drive
df.to_csv(DS_FILE)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129727 entries, 0 to 129726
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype              
---  ------                 --------------   -----              
 0   time                   129727 non-null  datetime64[ns, UTC]
 1   avg_gas_price          129716 non-null  float64            
 2   min_gas_price          129716 non-null  float64            
 3   max_gas_price          129716 non-null  float64            
 4   avg_base_fee_per_gas   129727 non-null  float64            
 5   avg_difficulty         129727 non-null  float32            
 6   avg_gas_limit          129727 non-null  float64            
 7   avg_gas_used           129727 non-null  float64            
 8   avg_size               129727 non-null  float64            
 9   avg_transaction_count  129727 non-null  float64            
 10  sum_tx_ether           129716 non-null  float64            
 11  num_blk                129727 non-null 

In [11]:
df.tail()

,time,avg_gas_price,min_gas_price,max_gas_price,avg_base_fee_per_gas,avg_difficulty,avg_gas_limit,avg_gas_used,avg_size,avg_transaction_count,sum_tx_ether,num_blk,num_trn
129722,2021-11-30 23:55:00+00:00,124.549098,101.833220,452.547873,110.820541,1.157759e+16,2.999999e+07,1.299718e+07,59.783143,130.285714,1140.048050,7,912.0
129723,2021-11-30 23:56:00+00:00,126.749791,109.914606,511.149332,114.727834,1.158246e+16,2.997071e+07,2.144985e+07,103.212333,264.666667,775.291316,3,794.0
129724,2021-11-30 23:57:00+00:00,142.455067,128.703035,968.287944,132.251091,1.156881e+16,2.994215e+07,1.669652e+07,90.444250,279.250000,1625.375807,4,1117.0
129725,2021-11-30 23:58:00+00:00,149.557319,135.158363,638.000000,137.185921,1.155692e+16,2.989834e+07,1.749532e+07,83.572000,233.000000,913.205329,2,466.0
129726,2021-11-30 23:59:00+00:00,135.709464,101.117227,465.646854,120.175149,1.155614e+16,2.995047e+07,1.161743e+07,60.122571,163.428571,947.064009,7,1144.0
